<a href="https://colab.research.google.com/github/zhangxs131/Text_Classification/blob/main/run_glue_for_thucnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##使用run_glue.py进行THUCNEWS的标题文本分类


In [ ]:
!pip install transformers datasets

In [2]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 96131, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 96131 (delta 117), reused 154 (delta 84), pack-reused 95922
Receiving objects: 100% (96131/96131), 88.98 MiB | 14.65 MiB/s, done.
Resolving deltas: 100% (70646/70646), done.


这里需要把transformers/examples/pytorch/text-classification/run_glue.py 文件进行一点修改

就是transfoerms版本检测，一般在50行左右，因为原来代码版本为测试版4.19.0dev0，我们下载是正式版4.18，不修改会报错。

In [ ]:
#下载数据
from datasets import load_dataset

cnews=load_dataset('seamew/THUCNewsTitle')

cnews['train'].to_csv('train.csv',index=False)
cnews['validation'].to_csv('valid.csv',index=False)
cnews['test'].to_csv('test.csv',index=False)

In [ ]:
#先用个bert-base-chinese试试水
!python transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path bert-base-chinese \
  --train_file train.csv \
  --validation_file valid.csv\
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 16 \
  --fp16 \
  --learning_rate 2e-5 \
  --save_strategy epoch \
  --num_train_epochs 3 \
  --output_dir /tmp/classifier/

##bert-base-chinese 跑了3轮，得到96的acc，效果确实已经很不错了

In [ ]:
#再试试bert-wwm-ext 哈工大hfl的那个模型
!python transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path hfl/chinese-bert-wwm-ext \
  --train_file train.csv \
  --validation_file valid.csv\
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 16 \
  --fp16 \
  --learning_rate 2e-5 \
  --save_strategy epoch \
  --num_train_epochs 3 \
  --output_dir /tmp/bert-wwm-ext/

#使用bert-wwm-ext准确率比上面高了4个千分点，96.4左右acc，还算可以吧

In [5]:
!pip install rjieba

     |████████████████████████████████| 3.8 MB 28.8 MB/s 


In [6]:
#再试试woberta，苏神他们公司的，但hf上下载太少，感觉可能是假的
!python transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path junnyu/wobert_chinese_plus_base \
  --train_file train.csv \
  --validation_file valid.csv\
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 16 \
  --fp16 \
  --learning_rate 2e-5 \
  --save_strategy epoch \
  --num_train_epochs 3 \
  --output_dir /content/woberta/

05/21/2022 02:11:50 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
05/21/2022 02:11:50 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=Non

###这wobert
跑了3轮，你准确率0.47？，肯定是huggingface上模型是broken的

In [ ]:
#再试试mac-bert，看在huggingface上用的挺多的
!python transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path hfl/chinese-macbert-base \
  --train_file train.csv \
  --validation_file valid.csv\
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 16 \
  --fp16 \
  --learning_rate 2e-5 \
  --save_strategy epoch \
  --num_train_epochs 3 \
  --output_dir /content/macbert/

###MacBert acc 0.963
挺好的了，看来macbert将mask替换为近义词，来解决pretrain和fine-tune不一致问题，效果还不凑合吧。

#再Roberta，精调的bert，感觉应该算是效果应该还行吧

跑了3轮 0.9638，可能模型大一些，需要跑的step多一些可能会更好。


In [ ]:
!python transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path hfl/chinese-roberta-wwm-ext \
  --train_file train.csv \
  --validation_file valid.csv\
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 16 \
  --fp16 \
  --learning_rate 2e-5 \
  --save_strategy epoch \
  --num_train_epochs 3 \
  --output_dir /content/roberta/